# Deplot notebook
> Creating an interactive environment for comparing stocks 



In [ ]:
#| default_exp deploy_project

In [ ]:
#| export

import buy_and_hold_vs_arima
import yfinance as yf
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.express as px
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc


In [ ]:
#| export

from buy_and_hold_vs_arima.arima_stats import create_auto_arima_prediction_future_2
from buy_and_hold_vs_arima.arima_stats import create_dataframe_with_series

In [ ]:
#| hide 

# how to import packages ! 
from buy_and_hold_vs_arima import arima_stats
arima_stats.foo("barak")

'Hello HAHAHAHbarak'

In [ ]:
#help(buy_and_hold_vs_arima)
#buy_and_hold_vs_arima.foo("hba")
dir(buy_and_hold_vs_arima)
#dir(np)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'arima_stats']

In [ ]:
barak = [1,2,3]
diker = [3,4,5]

In [ ]:
#| export 


#companies = ['AMZN','NFLX','GOOG']
companies = [
    "LUMI.TA",
    "DSCT.TA",
    "BEZQ.TA",
    "CEL.TA",
    "ESLT.TA",
    "NICE.TA",
    "TEVA.TA",
    "POLI.TA",
    "MZTF.TA",
    "FIBI.TA",
    "HARL.TA",
    "MGDL.TA",
    "CLIS.TA",
    "PHOE.TA",
    "MMHD.TA",
    "DRS.TA",
    "BSEN.TA",
    "HLAN.TA",
    "FTAL.TA",
    "DANE.TA",
    "ONE.TA",
    "MTRX.TA",
    "ALHE.TA",
    "UWAY.TA",
    "ICL.TA",
    "TA35.TA",
    "TA90.TA",
]

#companies = companies + ticker_stocks



tickers = [yf.Ticker(ticker).history( start='2020-12-10')['High'].rename(ticker) for ticker in companies]
df = pd.concat(tickers, axis=1)
df

for a in df.columns:
    print(a)
    
df[['LUMI.TA','ICL.TA']]

In [ ]:
#| export

# http://127.0.0.1:8050/

#df = px.data.gapminder()
#df.columns



external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        dcc.Markdown('''
        # Comparing Arima model and Buy and Hold strategy
        This is final project of master's degree written by yeshurun ben avraham 
        '''),
        dcc.Dropdown(
            id="dpdn2",
            value=["ICL.TA", "LUMI.TA"],
            multi=True,
            options=[{"label": x, "value": x} for x in df.columns],
        ),
         dbc.Row(
            
               # dcc.Graph(id="pie-graph", figure={}, className="six columns"),
                dcc.Graph(
                    id="my-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
           ),
            dbc.Row(
                dcc.Graph(
                    id="basic-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
            ),
        dbc.Row(
            # Values is the initial values
            # first arg and second arg are the overall range , third is the jumps 
                dcc.RangeSlider(0, len(df) - 1 , 1, count=1, value=[0, len(df) - 1] , id="range-inference") 
        ),
        dbc.Row(
            dcc.Graph(
                    id="inference-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
        ),
        
        
    ]
)


@app.callback(
    Output(component_id="my-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    print(country_chosen)
    
    dff = df[country_chosen]
    predication_data = pd.concat([create_dataframe_with_series(create_auto_arima_prediction_future_2 , dff[country]).rename(columns={"High": country, "pred": country +"pred"}) for country in country_chosen])
   
    fig = px.line(
        predication_data,
        #dff,
        #x="Date[Days]",
        #y="Stock[agora]",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig

@app.callback(
    Output(component_id="inference-graph", component_property="figure"),
    Input('range-inference', 'value'),
    Input(component_id="dpdn2", component_property="value"),
)
def update_output(slider_value,country_chosen):
    #print(value)
    dff = df[country_chosen]
    first_date_infer = dff.index[slider_value[0]]
    last_date_infer = dff.index[slider_value[1]]
    predication = pd.DataFrame()
    #concated_data = dff[country][first_date_infer:last_date_infer].copy()
    predication = pd.concat([create_dataframe_with_series(create_auto_arima_prediction_future_2 , dff[country][first_date_infer:last_date_infer].copy()).rename(columns={"High": country, "pred": country +"pred"}) for country in country_chosen])
    #predication.drop(country_chosen, axis=1)
    for country in country_chosen:
        predication[country + "True"] = dff[country][last_date_infer:]
    fig_global = px.line(
        predication,
        #x="Dates",
        #y=["Global Minimum Reg", "High"],
        #hover_data={"Dates": "|%B %d, %Y"},
        title="Inference BackTester",
    )

    fig_global.add_vline(
        x=first_date_infer, line_dash="dash", line_color="Blue"
    )
    fig_global.add_annotation(x=first_date_infer, text=str(first_date_infer))

    fig_global.add_vline(
        x=last_date_infer, line_dash="dash", line_color="Blue"
    )
    fig_global.add_annotation(x=last_date_infer, text=str(last_date_infer))
    
    
    
    
    
    return fig_global


@app.callback(
    Output(component_id="basic-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    
    dff = df[country_chosen]
    fig = px.line(
        dff,
        #dff,
        #x="Date[Days]",
        #y="Stock[agora]",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig






In [ ]:
#| hide
#if __name__ == "__main__":
#    app.run_server(debug=False)


In [ ]:
#| export 

def run_server(port=8050):
    from waitress import serve
    serve(app.server, host="0.0.0.0", port=port, threads=2)


In [ ]:
#| export
if __name__ == "__main__":
    #app.run_server(debug=False)
    from waitress import serve
    serve(app.server, host="0.0.0.0", port=8050, threads=2)


In [ ]:
import nbdev; nbdev.nbdev_export()